In [1]:
# ! pip install gensim
import gensim
import gensim.downloader as api
import numpy as np

In [2]:
from human_ai.code.dataset import get_train_dataset,get_test_dataset 

In [3]:
data = get_train_dataset(test_size=0.15)
train_data, test_data = data['train'], data['test']

Stringifying the column:   0%|          | 0/4000 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [4]:
X_train = [gensim.utils.simple_preprocess(sample['text']) for sample in train_data]
X_test = [gensim.utils.simple_preprocess(sample['text']) for sample in test_data]

In [5]:
w2v_model = gensim.models.Word2Vec(X_train,
                                   vector_size=1000,
                                   window=5,
                                   min_count=2)

In [6]:
w2v_model.wv.most_similar('king')

[('division', 0.9997978210449219),
 ('japan', 0.9996240735054016),
 ('foundation', 0.9996199607849121),
 ('polish', 0.9996166825294495),
 ('street', 0.9996074438095093),
 ('field', 0.9996019601821899),
 ('formed', 0.9995959401130676),
 ('democratic', 0.9995874762535095),
 ('defeating', 0.9995718002319336),
 ('soviet', 0.9995706081390381)]

In [7]:
words = set(w2v_model.wv.index_to_key)
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_train])
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_test])

/tmp/ipykernel_2287530/2953756858.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
/tmp/ipykernel_2287530/2953756858.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])


In [8]:
# Compute sentence vectors by averaging the word vectors for the words contained in the sentence
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(1000, dtype=float))
        
X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

In [9]:
# Are our sentence vector lengths consistent?
for i, v in enumerate(X_train_vect_avg):
    print(i, len(v))

0 1000
1 1000
2 1000
3 1000
4 1000
5 1000
6 1000
7 1000
8 1000
9 1000
10 1000
11 1000
12 1000
13 1000
14 1000
15 1000
16 1000
17 1000
18 1000
19 1000
20 1000
21 1000
22 1000
23 1000
24 1000
25 1000
26 1000
27 1000
28 1000
29 1000
30 1000
31 1000
32 1000
33 1000
34 1000
35 1000
36 1000
37 1000
38 1000
39 1000
40 1000
41 1000
42 1000
43 1000
44 1000
45 1000
46 1000
47 1000
48 1000
49 1000
50 1000
51 1000
52 1000
53 1000
54 1000
55 1000
56 1000
57 1000
58 1000
59 1000
60 1000
61 1000
62 1000
63 1000
64 1000
65 1000
66 1000
67 1000
68 1000
69 1000
70 1000
71 1000
72 1000
73 1000
74 1000
75 1000
76 1000
77 1000
78 1000
79 1000
80 1000
81 1000
82 1000
83 1000
84 1000
85 1000
86 1000
87 1000
88 1000
89 1000
90 1000
91 1000
92 1000
93 1000
94 1000
95 1000
96 1000
97 1000
98 1000
99 1000
100 1000
101 1000
102 1000
103 1000
104 1000
105 1000
106 1000
107 1000
108 1000
109 1000
110 1000
111 1000
112 1000
113 1000
114 1000
115 1000
116 1000
117 1000
118 1000
119 1000
120 1000
121 1000
122 1000
123

### Predict

In [13]:
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

clf = XGBClassifier()
clf.fit(X_train_vect_avg, train_data['label'])
predictions = clf.predict(X_test_vect_avg)
accuracy_score(test_data['label'], predictions)

0.5883333333333334